In [2]:
# %% [code]
import duckdb
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive/')

# ----------------------------------------------------------
# 1. DuckDBに接続し、data_mart_wideを取得
# ----------------------------------------------------------
db_path = "/content/drive/MyDrive/Colab Notebooks/standard/rossmann-store-sales/data/dwh.duckdb"
con = duckdb.connect(db_path)

print(f"[INFO] Connected to DuckDB at: {db_path}")

# data_mart_wideをPandas DataFrameで取得
df = con.execute("SELECT * FROM data_mart_wide").fetchdf()
con.close()
print("[INFO] Fetched data_mart_wide. Shape:", df.shape)

# ----------------------------------------------------------
# 2. データ確認 & 事前処理
# ----------------------------------------------------------
# このベースラインでは「分析に必要そうなカラムだけを選ぶ」程度で、新規特徴量は作りません。
# ただし、XGBoostに食わせるための"カテゴリ列のエンコード"は最小限行う必要あり。

# ざっくりカラムを確認
print("[INFO] Columns:", df.columns.tolist())

# ターゲット: Sales (売上)
# 入力特徴量: ここでは、'Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month' などに限定
# ※ 'Customers' を含むかどうかはケースバイケース。売上予測に顧客数を入れると「漏洩」に近くなる場合もあるが、
#   ここはベースラインなので特に制限せず使ってみる例とする。

use_cols = [
    "Store",
    "DayOfWeek",
    "Promo",
    "StateHoliday",
    "SchoolHoliday",
    "Year",
    "Month",
    # "Customers",  # 顧客数も入れるならコメントアウト解除
    # "Open",       # 必要なら追加
    # ... 必要に応じて既存カラムを追加
]

# ※日付(Date)やSalesはモデルに直接食わせるわけではなく、
#   "Sales" が教師ラベル、「Date」は学習データ/テストデータの分割にのみ使用。

# 欠損や型をチェック
print(df[use_cols + ["Sales","Date"]].info())

# Salesが0や欠損の行をどうするか？
# テストではそのまま扱うか、あるいはOpen=0の行を除外するなど。
# ベースラインなので簡単に:
df = df.dropna(subset=["Sales"])  # Sales欠損があれば除去

# ----------------------------------------------------------
# 3. 学習データとテストデータの分割 (時系列ベース)
# ----------------------------------------------------------
# 例: 2015-06-01より後をテストとする
df["Date"] = pd.to_datetime(df["Date"])
train_df = df[df["Date"] < "2015-06-01"].copy()
test_df = df[df["Date"] >= "2015-06-01"].copy()

print("[INFO] train_df shape:", train_df.shape, "  test_df shape:", test_df.shape)

# ----------------------------------------------------------
# 4. カテゴリ変換 (簡易なLabel Encoding)
# ----------------------------------------------------------
# XGBoostは文字列カテゴリを直接扱えないので、一旦Label Encoding等で数値化。
# (One-Hotなどでも構いませんが、ベースラインなのでシンプルに)
# scikit-learnのLabelEncoderを使いますが、StateHolidayなどは"none"含め複数種類がある。
# DayOfWeekはすでに int or categoryなら castすればOK。

from sklearn.preprocessing import LabelEncoder

def label_encode_cols(df, cols):
    for col in cols:
        df[col] = df[col].astype(str)
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df

categorical_cols = []
# 文字列型やcategory型のカラムを列挙(適宜変更)
# 例: 'StateHoliday' がobject(文字列)なら、適宜
if train_df["StateHoliday"].dtype == object:
    categorical_cols.append("StateHoliday")
if train_df["DayOfWeek"].dtype == object:
    categorical_cols.append("DayOfWeek")
# Storeは連番のはず(1~1115)なのでそのままでもOKだが、
# stringかどうかチェックして必要なら追加
if train_df["Store"].dtype == object:
    categorical_cols.append("Store")

train_df = label_encode_cols(train_df, categorical_cols)
test_df = label_encode_cols(test_df, categorical_cols)

# ----------------------------------------------------------
# 5. 学習用データセット作成
# ----------------------------------------------------------
X_train = train_df[use_cols]
y_train = train_df["Sales"].values

X_test = test_df[use_cols]
y_test = test_df["Sales"].values

print("[INFO] X_train shape:", X_train.shape, " y_train shape:", y_train.shape)
print("[INFO] X_test  shape:", X_test.shape, " y_test  shape:", y_test.shape)

# ----------------------------------------------------------
# 6. XGBoostでベースライン学習
# ----------------------------------------------------------
# 特徴量生成やハイパラチューニングは一切しない(=低精度OK)

model = xgb.XGBRegressor(
    # tree_method='hist',  # Colabのメモリ節約例(必要なら設定)
    random_state=42
)

model.fit(X_train, y_train)
print("[INFO] XGBoost training completed.")

# ----------------------------------------------------------
# 7. 予測＆評価 (RMSE, MAE)
# ----------------------------------------------------------
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"[RESULT] Baseline RMSE: {rmse:.2f}")
print(f"[RESULT] Baseline MAE : {mae:.2f}")

# あくまでベースラインなので、精度は低めに出てもOK。
# この後、特徴量エンジニアリングやハイパラ調整で改善を図る想定。


Mounted at /content/drive/
[INFO] Connected to DuckDB at: /content/drive/MyDrive/Colab Notebooks/standard/rossmann-store-sales/data/dwh.duckdb
[INFO] Fetched data_mart_wide. Shape: (560478, 23)
[INFO] Columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'WeekOfYear', 'store_key', 'date_key']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560478 entries, 0 to 560477
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Store          560478 non-null  int64         
 1   DayOfWeek      560478 non-null  int64         
 2   Promo          560478 non-null  int64         
 3   StateHoliday   560478 non-null  object        
 4   SchoolHoliday  560478 non-null  